# Com base nas dicas

# http://www.dsplog.com/2007/11/06/symbol-error-rate-for-4-qam/

In [1]:
# Import all necessary libraries.
import numpy as np
from scipy.special import erfc
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [2]:
# Number of QPSK symbols to be transmitted.
N = 1000000

In [3]:
def mod(V):

    ip = np.zeros((len(V),1),dtype=complex)
    inc = 0
    for v in V:
        if(v==0):
            ip[inc] = -1.0 - 1j*1.0
        elif(v==1):
            ip[inc] = -1.0 + 1j*1.0
        elif(v==2):
            ip[inc] = 1.0 - 1j*1.0
        else:
            ip[inc] = 1.0 + 1j*1.0
        inc += 1
    # Normalization of energy to 1.
    s = (1/np.sqrt(2))*ip;
    return s


def demod(c):
    c_seq = mod(np.array([0,1,2,3]))
    c_seq = c_seq[:,0]
    e = []
    for i in range(0,len(c_seq)):
        e.append(np.abs(c_seq[i] - c)**2)
    return e.index(np.min(e))


In [ ]:
# Create Es/N0 vector.
EsN0dB = np.arange(-2,22,2)

SER_simu = np.zeros(len(EsN0dB))
SER_theo = np.zeros(len(EsN0dB))
for idx in range(0,len(EsN0dB)):
   
    print('Es/N0 dB:', EsN0dB[idx])
   
    EsN0Lin = 10.0**(-(EsN0dB[idx]/10.0))
   
    # Generate N binary symbols.
    V = np.random.randint(0,4,(N,1))
   
    # Modulate binary stream into QPSK symbols.
    s = mod(V)
   
    # Generate noise vector with unitary variance.
    noise = np.sqrt(1.0/2.0)*(np.random.randn(N, 1) + 1j*np.random.randn(N, 1))
   
    # Pass symbols through AWGN channel.
    y = s + np.sqrt(EsN0Lin)*noise
   
    # Split array into random train and test subsets.
    s_test, s_train, y_test, y_train, v_test, v_train = train_test_split(s, y, V, random_state=42)
   
    Y = np.c_[y_train.real,y_train.imag]
       
    # Fit models for real and imaginary parts.
    km = KMeans(n_clusters=4, random_state=42)
    km.fit(Y)
   
    # Prediction.
    detected = km.predict(np.c_[y_test.real,y_test.imag])  
       
    # Simulated QPSK BER.
    error = 0
    for i in range(0, len(v_test)):
        pred = km.cluster_centers_[detected[i],0] + 1j*km.cluster_centers_[detected[i],1]
        d = demod(pred)
        if((v_test[i,0] != d)==True):
            error = error + 1
   
    SER_simu[idx] = 1.0 * error / len(v_test)
    
    # Theoretical QPSK BER.    
    SER_theo[idx] = erfc(np.sqrt(0.5*(10.0**(EsN0dB[idx]/10.0)))) - (1/4)*(erfc(np.sqrt(0.5*(10.0**(EsN0dB[idx]/10.0)))))**2.0

Es/N0 dB: -2
Es/N0 dB: 0
Es/N0 dB: 2
Es/N0 dB: 4


In [ ]:
# Print values of Es/N0 vs. SER.
for i in range(0,len(SER_simu)): print('Es/N0:%d \t- SER: %e' % (EsN0dB[i], SER_simu[i]))

In [ ]:
# Plot Es/N0 versus SER values.
fig = plt.figure(figsize=(12,8))
plt.plot(EsN0dB, SER_theo, 'g', label='theoretical')
plt.plot(EsN0dB, SER_simu, 'bo', label='simulated')
plt.xscale('linear')
plt.yscale('log')
plt.xlabel('Es/N0 [dB]')
plt.ylabel('SER')
plt.grid(True)
plt.title('QPSK detection with KMeans')
plt.legend()
plt.xlim([-2, 14])
plt.ylim([1e-5, 1])
plt.show()